## Problems and solutions of memory leaks

About resetting default graph:

- https://stackoverflow.com/questions/33765336/remove-nodes-from-graph-or-fa-entire-default-graph
- https://stackoverflow.com/documentation/tensorflow/3883/how-to-debug-a-memory-leak-in-tensorflow/13426/use-graph-finalize-to-catch-nodes-being-added-to-the-graph#t=201609011401186013951

About copying values of a tensorflow variable to another tensorflow variable

- https://stackoverflow.com/questions/33717772/how-can-i-copy-a-variable-in-tensorflow

Print value tensorflow variable
- https://stackoverflow.com/questions/33633370/how-to-print-the-value-of-a-tensor-object-in-tensorflow

#### Let us beggin

The most important rule for not increamenting a tf.graph is

- Never do sess.run(tf.operation) for example sess.run(tf.assign) etc...

Given two tensorflow variables `a` and `b` it's very different

- `sess.run(tf.add(a,b))`

than 

- `sess.run(add_a_and_b)`


In [79]:
import tensorflow as tf
import numpy as np
from pprint import pprint

### Example 1: sess.run(tf.add(a, b)))

This example adds nodes in the graph (see it ends up with 19 nodes)

In [94]:
tf.reset_default_graph()
a = tf.Variable(1, name='a')
pprint(tf.get_default_graph().get_operations())
print(len(tf.get_default_graph().get_operations()))

[<tf.Operation 'a/initial_value' type=Const>,
 <tf.Operation 'a' type=VariableV2>,
 <tf.Operation 'a/Assign' type=Assign>,
 <tf.Operation 'a/read' type=Identity>]
4


In [95]:
tf.reset_default_graph()
a = tf.Variable(1, name='a')
b = tf.Variable(2, name='b')
print(len(tf.get_default_graph().get_operations()))

8


In [96]:
%%time 
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(10):
        print( sess.run(tf.add(a, b)), end=" ")
    print("\n")

3 3 3 3 3 3 3 3 3 3 

CPU times: user 24 ms, sys: 0 ns, total: 24 ms
Wall time: 26.4 ms


In [97]:
print(len(tf.get_default_graph().get_operations()))

19


In [99]:
# Look here there are multiple operations:
# Add, Add_1, Add_2, .... Add_9 -> We only needed a single one!
tf.get_default_graph().get_operations()

[<tf.Operation 'a/initial_value' type=Const>,
 <tf.Operation 'a' type=VariableV2>,
 <tf.Operation 'a/Assign' type=Assign>,
 <tf.Operation 'a/read' type=Identity>,
 <tf.Operation 'b/initial_value' type=Const>,
 <tf.Operation 'b' type=VariableV2>,
 <tf.Operation 'b/Assign' type=Assign>,
 <tf.Operation 'b/read' type=Identity>,
 <tf.Operation 'init' type=NoOp>,
 <tf.Operation 'Add' type=Add>,
 <tf.Operation 'Add_1' type=Add>,
 <tf.Operation 'Add_2' type=Add>,
 <tf.Operation 'Add_3' type=Add>,
 <tf.Operation 'Add_4' type=Add>,
 <tf.Operation 'Add_5' type=Add>,
 <tf.Operation 'Add_6' type=Add>,
 <tf.Operation 'Add_7' type=Add>,
 <tf.Operation 'Add_8' type=Add>,
 <tf.Operation 'Add_9' type=Add>]

### Example 1: sess.run(compute_addition_a_b)

This example does not add nodes in the graph (well, only one, see it ends up with 10 nodes).

In [109]:
tf.reset_default_graph()
a = tf.Variable(1, name='a')
b = tf.Variable(2, name='b')
compute_addition_a_b = tf.add(a, b)

In [110]:
print(len(tf.get_default_graph().get_operations()))

9


In [111]:
%%time 
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(10):
        print(sess.run(compute_addition_a_b), end=" ")
    print("\n")

3 3 3 3 3 3 3 3 3 3 

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 7.31 ms


In [113]:
print(len(tf.get_default_graph().get_operations()))

10


In [114]:
tf.get_default_graph().get_operations()

[<tf.Operation 'a/initial_value' type=Const>,
 <tf.Operation 'a' type=VariableV2>,
 <tf.Operation 'a/Assign' type=Assign>,
 <tf.Operation 'a/read' type=Identity>,
 <tf.Operation 'b/initial_value' type=Const>,
 <tf.Operation 'b' type=VariableV2>,
 <tf.Operation 'b/Assign' type=Assign>,
 <tf.Operation 'b/read' type=Identity>,
 <tf.Operation 'Add' type=Add>,
 <tf.Operation 'init' type=NoOp>]

##  Example 2: sess.run(tf_variable.assign )


running tf.variable.assign several times is quite ineficcient since every time tf_variable.assign is called the graph grows.

In [161]:
tf.reset_default_graph()
n_his = 400
n_vis = 4

tf.reset_default_graph()
var_history = tf.get_variable("var_history", 
                              dtype=np.float32, 
                              initializer=np.array([range(0,400)], "float32")) 

var_history_shift = tf.get_variable("var_history_shift", 
                              dtype=np.float32, shape=(1, n_his-n_vis)) 


In [162]:
print(len(tf.get_default_graph().get_operations()))

15


In [163]:
%%time 
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # Graph is read-only after this statement. If uncommented then the code crashes
    #sess.graph.finalize()  

    for i in range(10):
        sess.run(var_history_shift.assign(var_history[0:1, n_vis:n_his]))
        sess.run(var_history[0:1, 0:n_his - n_vis].assign(var_history_shift))
        current_vis_vec = np.array([[i,i,i,i]], dtype="float32")
        sess.run(var_history[0:1, n_his-n_vis:n_his].assign(current_vis_vec))       
    print(sess.run(var_history))

[[  40.   41.   42.   43.   44.   45.   46.   47.   48.   49.   50.   51.
    52.   53.   54.   55.   56.   57.   58.   59.   60.   61.   62.   63.
    64.   65.   66.   67.   68.   69.   70.   71.   72.   73.   74.   75.
    76.   77.   78.   79.   80.   81.   82.   83.   84.   85.   86.   87.
    88.   89.   90.   91.   92.   93.   94.   95.   96.   97.   98.   99.
   100.  101.  102.  103.  104.  105.  106.  107.  108.  109.  110.  111.
   112.  113.  114.  115.  116.  117.  118.  119.  120.  121.  122.  123.
   124.  125.  126.  127.  128.  129.  130.  131.  132.  133.  134.  135.
   136.  137.  138.  139.  140.  141.  142.  143.  144.  145.  146.  147.
   148.  149.  150.  151.  152.  153.  154.  155.  156.  157.  158.  159.
   160.  161.  162.  163.  164.  165.  166.  167.  168.  169.  170.  171.
   172.  173.  174.  175.  176.  177.  178.  179.  180.  181.  182.  183.
   184.  185.  186.  187.  188.  189.  190.  191.  192.  193.  194.  195.
   196.  197.  198.  199.  200.  201. 

In [156]:
print(len(tf.get_default_graph().get_operations()))

176


##  Example 2: sess.run(tf_variable_assign)

In [167]:
tf.reset_default_graph()
n_his = 400
n_vis = 4

var_history = tf.get_variable("var_history", 
                              dtype=np.float32, 
                              initializer=np.array([range(0,n_his)], "float32")) 

var_visible = tf.get_variable("var_visible", 
                              dtype=np.float32, 
                              initializer=np.array([[9,9,9,9]], "float32")) 

var_history_shift = tf.get_variable("var_history_shift", 
                              dtype=np.float32, shape=(1, n_his-n_vis)) 

assign_var_history_shift  = var_history_shift.assign(var_history[0:1, n_vis:n_his])
update_history_with_shift  = var_history[0:1, 0:n_his - n_vis].assign(var_history_shift)
update_history_with_current_vis =  var_history[0:1, n_his-n_vis:n_his].assign(var_visible)


In [168]:
%%time
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.graph.finalize()  

    print(sess.run(var_visible))
    print(sess.run(var_history))
    print(sess.run(update_history_with_current_vis ))

[[ 9.  9.  9.  9.]]
[[   0.    1.    2.    3.    4.    5.    6.    7.    8.    9.   10.   11.
    12.   13.   14.   15.   16.   17.   18.   19.   20.   21.   22.   23.
    24.   25.   26.   27.   28.   29.   30.   31.   32.   33.   34.   35.
    36.   37.   38.   39.   40.   41.   42.   43.   44.   45.   46.   47.
    48.   49.   50.   51.   52.   53.   54.   55.   56.   57.   58.   59.
    60.   61.   62.   63.   64.   65.   66.   67.   68.   69.   70.   71.
    72.   73.   74.   75.   76.   77.   78.   79.   80.   81.   82.   83.
    84.   85.   86.   87.   88.   89.   90.   91.   92.   93.   94.   95.
    96.   97.   98.   99.  100.  101.  102.  103.  104.  105.  106.  107.
   108.  109.  110.  111.  112.  113.  114.  115.  116.  117.  118.  119.
   120.  121.  122.  123.  124.  125.  126.  127.  128.  129.  130.  131.
   132.  133.  134.  135.  136.  137.  138.  139.  140.  141.  142.  143.
   144.  145.  146.  147.  148.  149.  150.  151.  152.  153.  154.  155.
   156.  157.  158

In [325]:
print(len(tf.get_default_graph().get_operations()))

40


In [324]:
%%time 
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(10):
        #sess.run(var_history_shift.assign(var_history[0:1, n_vis:n_his]))
        sess.run(assign_var_history_shift)
        sess.run(update_history_with_shift)
        sess.run(update_history_with_current_vis)
        #current_vis_vec = np.array([[i,i,i,i]], dtype="float32")
    print(sess.run(var_history))

[[  40.   41.   42.   43.   44.   45.   46.   47.   48.   49.   50.   51.
    52.   53.   54.   55.   56.   57.   58.   59.   60.   61.   62.   63.
    64.   65.   66.   67.   68.   69.   70.   71.   72.   73.   74.   75.
    76.   77.   78.   79.   80.   81.   82.   83.   84.   85.   86.   87.
    88.   89.   90.   91.   92.   93.   94.   95.   96.   97.   98.   99.
   100.  101.  102.  103.  104.  105.  106.  107.  108.  109.  110.  111.
   112.  113.  114.  115.  116.  117.  118.  119.  120.  121.  122.  123.
   124.  125.  126.  127.  128.  129.  130.  131.  132.  133.  134.  135.
   136.  137.  138.  139.  140.  141.  142.  143.  144.  145.  146.  147.
   148.  149.  150.  151.  152.  153.  154.  155.  156.  157.  158.  159.
   160.  161.  162.  163.  164.  165.  166.  167.  168.  169.  170.  171.
   172.  173.  174.  175.  176.  177.  178.  179.  180.  181.  182.  183.
   184.  185.  186.  187.  188.  189.  190.  191.  192.  193.  194.  195.
   196.  197.  198.  199.  200.  201. 

In [267]:
print(len(tf.get_default_graph().get_operations()))

37


### Example 3: making the visible vector dynamic


A tensorflow variable can be fed dynamically- This can be handy for example for changing at run time the values of some variables in our graph. An example:

In [328]:
tf.reset_default_graph()
a = tf.Variable(1)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(a, feed_dict={a:5})) 
    print(sess.run(a, feed_dict={a:23}))

5
23


In [329]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(a)) 


1


In [120]:
tf.reset_default_graph()
n_his = 400
n_vis = 4

var_history = tf.get_variable("var_history", 
                              dtype=np.float32, 
                              initializer=np.array([range(0,n_his)], "float32")) 

var_visible = tf.get_variable("var_visible", 
                              dtype=np.float32, 
                              initializer=np.array([[9,9,9,9]], "float32")) 

var_history_shift = tf.get_variable("var_history_shift", 
                              dtype=np.float32, shape=(1, n_his-n_vis)) 

assign_var_history_shift  = var_history_shift.assign(var_history[0:1, n_vis:n_his])
update_history_with_shift  = var_history[0:1, 0:n_his - n_vis].assign(var_history_shift)
update_history_with_current_vis =  var_history[0:1, n_his-n_vis:n_his].assign(var_visible)


In [121]:
%%time 
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.graph.finalize()  # Graph is read-only after this statement.

    for i in range(10):
        #sess.run(var_history_shift.assign(var_history[0:1, n_vis:n_his]))
        sess.run(assign_var_history_shift)
        sess.run(var_history, feed_dict={var_visible: np.array([[i,i,i,i]], "float32")})
        sess.run(update_history_with_shift)
        sess.run(update_history_with_current_vis)
        #current_vis_vec = np.array([[i,i,i,i]], dtype="float32")
    print(sess.run(var_history))

[[  40.   41.   42.   43.   44.   45.   46.   47.   48.   49.   50.   51.
    52.   53.   54.   55.   56.   57.   58.   59.   60.   61.   62.   63.
    64.   65.   66.   67.   68.   69.   70.   71.   72.   73.   74.   75.
    76.   77.   78.   79.   80.   81.   82.   83.   84.   85.   86.   87.
    88.   89.   90.   91.   92.   93.   94.   95.   96.   97.   98.   99.
   100.  101.  102.  103.  104.  105.  106.  107.  108.  109.  110.  111.
   112.  113.  114.  115.  116.  117.  118.  119.  120.  121.  122.  123.
   124.  125.  126.  127.  128.  129.  130.  131.  132.  133.  134.  135.
   136.  137.  138.  139.  140.  141.  142.  143.  144.  145.  146.  147.
   148.  149.  150.  151.  152.  153.  154.  155.  156.  157.  158.  159.
   160.  161.  162.  163.  164.  165.  166.  167.  168.  169.  170.  171.
   172.  173.  174.  175.  176.  177.  178.  179.  180.  181.  182.  183.
   184.  185.  186.  187.  188.  189.  190.  191.  192.  193.  194.  195.
   196.  197.  198.  199.  200.  201. 

In [122]:
print(len(tf.get_default_graph().get_operations()))

36


Notice that this procedure did not modigy `var_history` as we could have expected.
We need to do an assign operation at runetime to do so.

In [175]:
tf.reset_default_graph()
n_his = 400
n_vis = 4

var_history = tf.get_variable("var_history", 
                              dtype=np.float32, 
                              initializer=np.array([range(0,n_his)], "float32")) 

var_visible = tf.get_variable("var_visible", 
                              dtype=np.float32, 
                              initializer=np.array([[9,9,9,9]], "float32")) 

placeholder_visible = tf.get_variable(name="placeholder_visible", 
                                      shape=(1,n_vis),
                                      dtype=np.float32) 

var_history_shift = tf.get_variable("var_history_shift", 
                                    dtype=np.float32, shape=(1, n_his-n_vis)) 

assign_var_history_shift  = var_history_shift.assign(var_history[0:1, n_vis:n_his])
update_history_with_shift  = var_history[0:1, 0:n_his - n_vis].assign(var_history_shift)
update_history_with_current_vis =  var_history[0:1, n_his-n_vis:n_his].assign(var_visible)
assign_var_visible = var_visible.assign(placeholder_visible)


In [176]:
%%time 
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.graph.finalize()  # Graph is read-only after this statement.

    for i in range(10):
        #sess.run(var_history_shift.assign(var_history[0:1, n_vis:n_his]))
        sess.run(assign_var_history_shift)
        sess.run([assign_var_visible], feed_dict={placeholder_visible: np.array([[i,i,i,i]], "float32")})
        sess.run(update_history_with_shift)
        sess.run(update_history_with_current_vis)
        #current_vis_vec = np.array([[i,i,i,i]], dtype="float32")
    print(sess.run(var_history))

[[  40.   41.   42.   43.   44.   45.   46.   47.   48.   49.   50.   51.
    52.   53.   54.   55.   56.   57.   58.   59.   60.   61.   62.   63.
    64.   65.   66.   67.   68.   69.   70.   71.   72.   73.   74.   75.
    76.   77.   78.   79.   80.   81.   82.   83.   84.   85.   86.   87.
    88.   89.   90.   91.   92.   93.   94.   95.   96.   97.   98.   99.
   100.  101.  102.  103.  104.  105.  106.  107.  108.  109.  110.  111.
   112.  113.  114.  115.  116.  117.  118.  119.  120.  121.  122.  123.
   124.  125.  126.  127.  128.  129.  130.  131.  132.  133.  134.  135.
   136.  137.  138.  139.  140.  141.  142.  143.  144.  145.  146.  147.
   148.  149.  150.  151.  152.  153.  154.  155.  156.  157.  158.  159.
   160.  161.  162.  163.  164.  165.  166.  167.  168.  169.  170.  171.
   172.  173.  174.  175.  176.  177.  178.  179.  180.  181.  182.  183.
   184.  185.  186.  187.  188.  189.  190.  191.  192.  193.  194.  195.
   196.  197.  198.  199.  200.  201. 

In [336]:
print(len(tf.get_default_graph().get_operations()))

47


### IMPORTANT
### Use Graph.finalize() to avoid nodes being added to the graph


The following comment is from - https://stackoverflow.com/documentation/tensorflow/3883/how-to-debug-a-memory-leak-in-tensorflow/13426/use-graph-finalize-to-catch-nodes-being-added-to-the-graph#t=201707170928086124668

The most common mode of using TensorFlow involves first building a dataflow graph of TensorFlow operators (like tf.constant() and tf.matmul(), then running steps by calling the tf.Session.run() method in a loop (e.g. a training loop).

**A common source of memory leaks is where the training loop contains calls that add nodes to the graph, and these run in every iteration, causing the graph to grow**. These may be obvious (e.g. a call to a TensorFlow operator like tf.square()), implicit (e.g. a call to a TensorFlow library function that creates operators like tf.train.Saver()), or subtle (e.g. a call to an overloaded operator on a tf.Tensor and a NumPy array, which implicitly calls tf.convert_to_tensor() and adds a new tf.constant() to the graph).

The tf.Graph.finalize() method can help to catch leaks like this: it marks a graph as read-only, and raises an exception if anything is added to the graph. For example:

```
loss = ...
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    sess.graph.finalize()  # Graph is read-only after this statement.

    for _ in range(1000000):
        sess.run(train_op)
        loss_sq = tf.square(loss)  # Exception will be thrown here.
        sess.run(loss_sq)
```

In this case, the overloaded * operator attempts to add new nodes to the graph:

```
loss = ...
# ...
with tf.Session() as sess:
    # ...
    sess.graph.finalize()  # Graph is read-only after this statement.
    # ...
    dbl_loss = loss * 2.0  # Exception will be thrown here
```

## Summary on using tf.Session(graph=g)
```
with tf.Session(graph=g) as sess:
    sess.run(tf.train.Saver().restore(sess,"saved_graph.ckpt"))
    sess.run(tf.global_variables_initializer())  # If you train the model for the first time
    sess.graph.finalize()                        # Graph is read-only after this statement.
```